In [64]:
import numpy as np
import pandas as pd

In [83]:
#order of calculation of f based on arcs
arc_f={(3,2):[(1,3)], 
       (3,4):[(1,3)],
       (5,2):[(1,5)], 
       (5,6):[(1,5)],
       (2,4):[(3,2),(1,2),(5,2)],
       (2,7):[(3,2),(1,2),(5,2)],
       (2,6):[(3,2),(1,2),(5,2)], 
       (4,7):[(2,4),(3,4)], 
       (6,7):[(2,6),(5,6)]
      } 

#tij
arc_tij={(1,2):10, 
         (1,3):20, 
         (1,5):25, 
         (2,4):35, 
         (2,6):35, 
         (2,7):30, 
         (3,2):15, 
         (3,4):20, 
         (4,7):30, 
         (5,2):20, 
         (5,6):25,  
         (6,7):10 
        }

#alpha, beta
node_albe={1:(0,np.inf), #replace np.inf by 12 in order to check the code
           2:(18,30), 
           3:(35,45), 
           4:(64,75), 
           5:(35,50), 
           6:(64,75), 
           7:(0,np.inf)
          }

#gamma, delta
arc_gamdel={(1,2):(10,15),
            (1,3):(10,25), 
            (1,5):(15,25), 
            (2,4):(28,35), 
            (2,6):(26,34), 
            (2,7):(32,60), 
            (3,2):(35,50), 
            (3,4):(43,50), 
            (4,7):(70,99), 
            (5,2):(30,80), 
            (5,6):(48,55), 
            (6,7):(72,80)
           }

In [84]:
#feasibility function
def feasib(fij,alph_i,beta_i,gam_ij,delta_ij):
    #conditions from (1) to (5) 
    if ((alph_i <= fij) and (fij<= beta_i)):#condition (1) #and (alph_i <= gam_ij)
        l_pi= max(0,gam_ij-fij)#lower pi
        u_pi=min(beta_i-fij,delta_ij-fij)#upper pi
        if l_pi <=u_pi:
            return True
        else:
            return False
    #conditions (2) and (4)
    elif ((beta_i<=fij) and (gam_ij<=fij) and (fij <= delta_ij)) or ((gam_ij<fij) and (fij<alph_i)and(alph_i<=delta_ij)) or ((gam_ij<fij)and (fij<delta_ij)and(delta_ij<=alph_i)):
        return True
    #conditions (3) and (5)
    elif ((fij<gam_ij)and (gam_ij<alph_i))or((fij<alph_i)and(alph_i<gam_ij)and(gam_ij<beta_i))or((fij<alph_i)and(alph_i<beta_i)and(beta_i<gam_ij))or(delta_ij<fij):
        return False

In [85]:
#calculation of fjk and check for feasibility of fjk
def cal_feasib_fjk(fij,alph_i,alph_j,beta_i,beta_j,gam_ij,delta_ij,gam_jk,delta_jk,tij): 
    f=None
    pi=None
    k=False
    #condition (1) 
    if ((alph_i <= fij) and (fij<= beta_i)): # and (alph_i <= gam_ij)
        l_pi= max(0,gam_ij-fij)#lower pi
        u_pi=min(beta_i-fij,delta_ij-fij)#upper pi
        if l_pi>u_pi:
            return f,pi,k
        else:
            for pi in range (l_pi,u_pi+1):
                fjk= fij+tij+pi
                if feasib(fjk,alph_j,beta_j,gam_jk,delta_jk) == True:
                    f=fjk
                    k=True
                    break
            return f,pi,k
    #conditions (2) and (4)
    elif ((beta_i<=fij) and (gam_ij<=fij) and (fij <= delta_ij)) or ((gam_ij<fij) and (fij<alph_i)and(alph_i<=delta_ij)) or ((gam_ij<fij)and (fij<delta_ij)and(delta_ij<=alph_i)):
        pi=0
        fjk= fij+tij+pi
        k=True
        return fjk,pi,k
    #conditions (3) and (5)
    elif ((fij<gam_ij)and (gam_ij<alph_i))or((fij<alph_i)and(alph_i<gam_ij)and(gam_ij<beta_i))or((fij<alph_i)and(alph_i<beta_i)and(beta_i<gam_ij))or(delta_ij<fij):
        k=False
        return f,pi,k

In [86]:
#dict of values of f
dict_f={(1,3):(0,10),(1,2):(0,10), (1,5):(0,15)}
k=0
N=9 #number of iterations for the for loop #we have 12-3=9 arcs to calculate
#feasib(fij,alph_i,beta_i,gam_ij,delta_ij)
for key in arc_f:
    k+=1
    #print(key[0],key[1])#j=key[0], k=key[1]
    print(key)
    prev_arc=arc_f[key]
    f_list=[]
    fpi_list=[]
    for arc in prev_arc:
        print(arc)
        fij=dict_f[arc][0]
        #print('fij=',fij)
        tij=arc_tij[arc]
        #print('tij=',tij)
        alph_i, beta_i= node_albe[arc[0]][0],node_albe[arc[0]][1]
        #print('alph_i, beta_i=',node_albe[arc[0]][0],node_albe[arc[0]][1])
        
        #alph_j, beta_j
        alph_j, beta_j= node_albe[key[0]][0],node_albe[key[0]][1]
        #print('alph_i, beta_i=',node_albe[arc[0]][0],node_albe[arc[0]][1])
        
        
        gam_ij,delta_ij=arc_gamdel[arc][0], arc_gamdel[arc][1]
        #print('gam_ij,delta_ij=',arc_gamdel[arc][0],  arc_gamdel[arc][1])
        
        #gam_jk,delta_jk
        gam_jk,delta_jk=arc_gamdel[key][0], arc_gamdel[key][1]
        
        #print(feasib(fij,alph_i,beta_i,gam_ij,delta_ij)) 
        
        #cal_feasib_fjk
        print(cal_feasib_fjk(fij,alph_i,alph_j,beta_i,beta_j,gam_ij,delta_ij,gam_jk,delta_jk,tij))
        f, pi, b= cal_feasib_fjk(fij,alph_i,alph_j,beta_i,beta_j,gam_ij,delta_ij,gam_jk,delta_jk,tij)
        #f_list.append()
        #append to dict of f_list
        if b==True:
            fpi_list.append((f, pi))
            f_list.append(f)
            
    #Dominance criteria (look for the min f)
    index_min = np.argmin(f_list)
    dict_f[key]=fpi_list [index_min]
    if k>=N:
        break
#set of outputs: all paths and their characteristics 
# (j, k)
# (i, j)
# (fjk, pi, feasibility[True or False])

(3, 2)
(1, 3)
(35, 15, True)
(3, 4)
(1, 3)
(35, 15, True)
(5, 2)
(1, 5)
(40, 15, True)
(5, 6)
(1, 5)
(40, 15, True)
(2, 4)
(3, 2)
(None, 10, False)
(1, 2)
(20, 10, True)
(5, 2)
(None, 10, False)
(2, 7)
(3, 2)
(50, 0, True)
(1, 2)
(None, 15, False)
(5, 2)
(60, 0, True)
(2, 6)
(3, 2)
(None, 10, False)
(1, 2)
(20, 10, True)
(5, 2)
(None, 10, False)
(4, 7)
(2, 4)
(64, 9, True)
(3, 4)
(64, 9, True)
(6, 7)
(2, 6)
(64, 9, True)
(5, 6)
(73, 8, True)


In [87]:
# set of outputs: all feasible paths  ?
# (start node, end node) : (fij, pi)
dict_f 

{(1, 3): (0, 10),
 (1, 2): (0, 10),
 (1, 5): (0, 15),
 (3, 2): (35, 15),
 (3, 4): (35, 15),
 (5, 2): (40, 15),
 (5, 6): (40, 15),
 (2, 4): (20, 10),
 (2, 7): (50, 0),
 (2, 6): (20, 10),
 (4, 7): (64, 9),
 (6, 7): (64, 9)}

In [ ]:
# Extract optimal feasible path in accordance to dominance criteria